In [1]:
# Libraries Import
import numpy as np
import matplotlib.pyplot as plt

# Class & Function import
from ql_core import Agent, Environment, train
from utils import plot_rewards_history, visualize_policy

In [2]:
# --- 1. Configuração do Ambiente e do Agente ---
GRID_SIZE = (10, 10)
START_STATE = (0, 0, 0)
GOAL_STATE = (9, 9, 0)
ACTIONS_TYPE = "omni"

# Instancia o ambiente
env = Environment(
    grid_size=GRID_SIZE,
    start=START_STATE,
    goal=GOAL_STATE,
    actions_type=ACTIONS_TYPE
)

# Instancia o agente
agent = Agent(
    state_shape=env.state_shape,
    n_actions=env.n_actions,
    learning_rate=0.1,
    discount_factor=0.99,
    exploration_rate=1.0,
    exploration_decay_rate=0.9999, # Decaimento mais lento para problemas mais complexos
    min_exploration_rate=0.01
)

# --- 2. Treinamento ---
print("Iniciando o treinamento do agente...")
# Aumentar n_episodes pode levar a melhores resultados, mas demora mais
rewards = train(agent, env, n_episodes=20000, max_steps_per_episode=100)
print("Treinamento concluído.")

# --- 3. Visualização ---
# Plota o histórico de recompensas
plot_rewards_history(rewards)

# Visualiza a política aprendida para a orientação inicial (psi = 0)
# Você pode mudar o último parâmetro para ver a política para outras orientações
visualize_policy(agent, env, psi_layer_to_show=0)

TypeError: Agent.__init__() got an unexpected keyword argument 'exploration_rate'